# Librerias

In [2]:
# Tratado de datos

import pandas as pd
import numpy as np
import sys

# Flujo 

from sklearn.pipeline import Pipeline

# Preprocesamiento

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Separacion de datos 

from sklearn.model_selection import train_test_split

# Balanceo de datos

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Escalado 

from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Modelo 

from sklearn.svm import SVC

# Evaluacion 

from sklearn.metrics import roc_auc_score, f1_score,accuracy_score,recall_score,classification_report,confusion_matrix

# visualizacion

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sys.path.append('../../src/utils')

from Pipeline_ import get_pipeline

In [4]:
df= pd.read_parquet('../../data/processed/telecom_limpio.parquet')

In [5]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [6]:
X= df.drop('Churn',axis=1)
y = df['Churn']

In [7]:
y = y.replace({'Yes': 1, 'No': 0});

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:


pipeline = get_pipeline()
pipeline.set_params(balanceo=RandomUnderSampler(random_state=42),escalado='passthrough',modelo = SVC(probability=True));

pipeline

Pipeline(steps=[('preproceso',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('label', OrdinalEncoder(),
                                                  ['MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaymentMethod']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'PaperlessBilling'])])),
                ('balanceo', RandomUnderSampler(random_state=42)),
                ('escalado', 'passthrough'),
                ('modelo', SVC(probability=True))])

In [10]:
pipeline.fit(X_train, y_train);

In [11]:
y_test_pred =pipeline.predict(X_test)
y_train_pred =pipeline.predict(X_train)

In [12]:
print(f'accuracy en prueba : {accuracy_score(y_test, y_test_pred)}')
print(f'accuracy en entrenamiento : {accuracy_score(y_train, y_train_pred)}')

accuracy en prueba : 0.6721078779276083
accuracy en entrenamiento : 0.6673766418175364


In [13]:
y_test_pred_Auc =pipeline.predict_proba(X_test)[:, 1]
y_train_pred_Auc =pipeline.predict_proba(X_train)[:, 1]

print(f'Auc-roc en prueba : {roc_auc_score(y_test, y_test_pred)}')
print(f'Auc-roc en entrenamiento : {roc_auc_score(y_train, y_train_pred)}')

Auc-roc en prueba : 0.6346227499042513
Auc-roc en entrenamiento : 0.6231198452842185


In [14]:
print(f'f1-Score en prueba : {f1_score(y_test, y_test_pred)}')
print(f'f1-Score en entrenamiento : {f1_score(y_train, y_train_pred)}')

f1-Score en prueba : 0.4726027397260274
f1-Score en entrenamiento : 0.45775462962962965
